In [143]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from sklearn.preprocessing import StandardScaler

import ast
import pandas as pd
import numpy as np

In [185]:
BATCH_SIZE = 10
EPOCH = 300
HIDDEN_SIZE = 4

In [186]:
df = pd.read_csv("dataset.csv")
data_target = np.asarray(df[["data", "target"]])
np.random.shuffle(data_target)

train_X = data_target[:85, 0]
train_y = data_target[:85, 1]

valid_X = data_target[85:, 0]
valid_y = data_target[85:, 1]

for i in range(train_X.shape[0]):
    train_X[i] = np.asarray(ast.literal_eval(train_X[i])).flatten()

for i in range(valid_X.shape[0]):
    valid_X[i] = np.asarray(ast.literal_eval(valid_X[i])).flatten()

train_X = np.array([x for x in train_X])
train_y = np.asarray(train_y.reshape((-1, 1)), dtype=np.float32)

valid_X = np.array([x for x in valid_X])
valid_y = np.asarray(valid_y.reshape((-1, 1)), dtype=np.float32)

ss = StandardScaler()
train_X = ss.fit_transform(train_X)
valid_X = ss.transform(valid_X)

In [187]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()

        self.x = x
        self.y = y

    def __getitem__(self, index):
        
        x = torch.Tensor(self.x[index])
        y = torch.Tensor(self.y[index])

        return (x, y)
    
    def __len__(self):
        count = self.x.shape[0]
        return count

train_dataset = CustomDataset(train_X, train_y)
valid_dataset = CustomDataset(valid_X, valid_y)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)


In [188]:
class MLP(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()

        self.fc = nn.Sequential(
            nn.Linear(30, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, 16),
            nn.LeakyReLU(),
            nn.Linear(16, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

In [194]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MLP(HIDDEN_SIZE).to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.AdamW(model.parameters())


for epoch in range(EPOCH):
    train_loss = 0.0
    valid_loss = 0.0
    predicted_labels = []
    true_labels = []
    
    model.train()
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        train_loss += loss.item() * inputs.size(0)


    model.eval()
    with torch.no_grad():
        for inputs, labels in valid_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            valid_loss += loss.item() * inputs.size(0)

    print(f"Epoch {epoch}")
    print(f"Average Training Loss: {train_loss / len(train_dataloader.sampler)}")
    print(f"Average Validation Loss: {valid_loss / len(valid_dataloader.sampler)}")
    # print(f"Validation Accuracy: {accuracy_score(true_labels, predicted_labels)}")

    # prfs = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')
    
    # print(f"Validation Precision: {prfs[0]}")
    # print(f"Validation Recall: {prfs[1]}")
    # print(f"Validation F1-Score: {prfs[2]}")

    print("==============================================================")    

Epoch 0
Average Training Loss: 3978.209185431985
Average Validation Loss: 4161.154398600261
Epoch 1
Average Training Loss: 3974.4668255974266
Average Validation Loss: 4157.700622558594
Epoch 2
Average Training Loss: 3971.2214211856617
Average Validation Loss: 4154.703470865886
Epoch 3
Average Training Loss: 3968.5004595588234
Average Validation Loss: 4152.0427652994795
Epoch 4
Average Training Loss: 3966.044017118566
Average Validation Loss: 4149.703715006511
Epoch 5
Average Training Loss: 3963.824146943934
Average Validation Loss: 4147.613301595052
Epoch 6
Average Training Loss: 3961.81107823989
Average Validation Loss: 4145.6853434244795
Epoch 7
Average Training Loss: 3959.907901539522
Average Validation Loss: 4143.8677978515625
Epoch 8
Average Training Loss: 3958.1335161994484
Average Validation Loss: 4142.092875162761
Epoch 9
Average Training Loss: 3956.370548023897
Average Validation Loss: 4140.321716308594
Epoch 10
Average Training Loss: 3954.6004997702207
Average Validation Loss

In [167]:
data = np.asarray([[321, 243, 7.279782772064209], [321, 243, 7.279782772064209], [321, 243, 7.279782772064209], [327, 217, 8.607392311096191], [327, 217, 8.607392311096191], [335, 195, 10.567272186279297], [335, 195, 10.567272186279297], [335, 195, 10.567272186279297], [335, 195, 10.567272186279297], [342, 182, 11.011177062988281]], dtype=np.float32)
data = ss.transform(data.flatten().reshape(1, -1))

model(torch.tensor(data).to(device))

tensor([[63.5088]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [168]:
torch.save(model, "model.pth")

In [169]:
from joblib import dump, load
dump(ss, 'std_scaler.bin', compress=True)

['std_scaler.bin']